In [23]:
import pandas as pd

In [34]:
df = pd.read_csv('airflow/data/Dataset/data.csv')
topics = pd.read_csv('airflow/data/Outputs/topics.csv')
trends = pd.read_csv('airflow/data/Outputs/trends.csv')
topics_columns = topics.columns.tolist()
trends_columns = trends.columns.tolist()
# print(topics_columns)
tuple(topics_columns)

('doc_num',
 'de,et,la,le,des',
 'de,rapport,hausse,période,2022',
 'pas,est,il,ne,pour',
 'milliards,de,trimestre,crédits,fin',
 'économie,banque,développement,de,la')

In [11]:
import psycopg2 as pg2

# Connect to database
conn = pg2.connect(host='localhost', database='optionsdata', user='openoptions', password='openoptions', port='5432')

In [12]:
conn.status

1

In [13]:
s = ""
s += "SELECT"
s += " table_schema"
s += ", table_name"
s += " FROM information_schema.tables"
s += " WHERE"
s += " ("
s += " table_schema = 'public'"
s += " )"
s += " ORDER BY table_schema, table_name;"

In [14]:
db_cursor = conn.cursor()
db_cursor.execute(s)
list_tables = db_cursor.fetchall()

for t_name_table in list_tables:
    print(t_name_table)

('public', 'articles')
('public', 'fb')
('public', 'goog')


In [19]:
# query database
query = 'SELECT * FROM articles;'

In [20]:
cur = conn.cursor()
cur.execute(query)

ha = cur.fetchall()
print(type(ha))

<class 'list'>


In [21]:
import pandas as pd
df = pd.DataFrame(ha)

In [22]:
df.head()

,0,1,2,3,4
0,Pêche. Hausse des débarquements de 14% à fin m...,Les débarquements des produits commercialisés ...,2022-06-14,https://www.challenge.ma/peche-hausse-des-deba...,https://www.challenge.ma/wp-content/uploads/20...
1,"Trésorerie. Le déficit budgétaire à 14,6 MMDH ...",La situation des charges et ressources du Trés...,2022-06-13,https://www.challenge.ma/tresorerie-le-deficit...,https://www.challenge.ma/wp-content/uploads/20...
2,Ouverture à Rabat de la réunion de haut niveau...,Les travaux de la réunion de haut niveau du co...,2022-06-13,https://www.challenge.ma/ouverture-a-rabat-de-...,https://www.challenge.ma/wp-content/uploads/20...
3,Maroc/Emirats Arabes Unis. Signature d’un MoU ...,La Confédération générale des entreprises du M...,2022-06-09,https://www.challenge.ma/maroc-emirats-arabes-...,https://www.challenge.ma/wp-content/uploads/20...
4,Prêts bancaires. Plus de 900 milliards de DH s...,"Selon Bank Al-Maghrib, l’encours du crédit ban...",2022-06-03,https://www.challenge.ma/banques-les-prets-acc...,https://www.challenge.ma/wp-content/uploads/20...
